# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [1]:
# Import the data
import pandas as pd

df = pd.read_csv('titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [2]:
# Total number of people who survived/didn't survive
df.duplicated().sum()
df['Name'].duplicated().sum() # no dups

df['Survived'].sum() #342

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [3]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = df[relevant_columns]
dummy_dataframe['Pclass'] = dummy_dataframe['Pclass'].astype('object')
# dummy_dataframe.info() 
dummy_dataframe = pd.get_dummies(dummy_dataframe, drop_first=True)

dummy_dataframe
dummy_dataframe.info()
dummy_dataframe.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Age           714 non-null float64
SibSp         891 non-null int64
Fare          891 non-null float64
Survived      891 non-null int64
Pclass_2      891 non-null uint8
Pclass_3      891 non-null uint8
Sex_male      891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(2), int64(2), uint8(5)
memory usage: 32.3 KB


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(891, 9)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [4]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 9)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [5]:
# Split the data into X and y
X = dummy_dataframe.drop(columns='Survived')
y = dummy_dataframe['Survived']

In [7]:
X
y

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 714, dtype: int64

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [8]:
# Build a logistic regression model using statsmodels

import statsmodels.api as sm

X = sm.add_constant(X)

logit_model = sm.Logit(y, X)

result = logit_model.fit()



Optimization terminated successfully.
         Current function value: 0.443266
         Iterations 6


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      705
Method:                           MLE   Df Model:                            8
Date:                Wed, 24 Feb 2021   Pseudo R-squ.:                  0.3437
Time:                        11:24:23   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.889e-67
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4362      0.534      8.302      0.000       3.389       5.484
Age           -0.0431      0.008     -5.190      0.000      -0.059      -0.027
SibSp         -0.3804      0.125     -3.041      0.002      -0.626      -0.135
Fare           0.0012      0.002      0.466      0.642      -0.004       0.006
Pclass_2      -1.2075      0.328     -3.686      0.000      -1.850      -0.565
Pclass_3      -2.4221      0.340     -7.129      0.000      -3.088      -1.756
Sex_male      -2.6233      0.218    -12.035      0.000      -3.051      -2.196
Embarked_Q    -0.8259      0.598     -1.381      0.167      -1.998       0.347
Embarked_S    -0.4137      0.274     -1.511      0.131      -0.951       0.123
==============================================================================
"""

## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [9]:
# Summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      705
Method:                           MLE   Df Model:                            8
Date:                Wed, 24 Feb 2021   Pseudo R-squ.:                  0.3437
Time:                        11:28:51   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.889e-67
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4362      0.534      8.302      0.000       3.389       5.484
Age           -0.0431      0.008     -5.190      0.000      -0.059      -0.027
SibSp         -0.3804      0.125     -3.041      0.002      -0.626      -0.135
Fare           0.0012      0.002      0.466      0.642      -0.004       0.006
Pclass_2      -1.2075      0.328     -3.686      0.000      -1.850      -0.565
Pclass_3      -2.4221      0.340     -7.129      0.000      -3.088      -1.756
Sex_male      -2.6233      0.218    -12.035      0.000      -3.051      -2.196
Embarked_Q    -0.8259      0.598     -1.381      0.167      -1.998       0.347
Embarked_S    -0.4137      0.274     -1.511      0.131      -0.951       0.123
==============================================================================
"""

In [ ]:
# Your comments here
# age and embarked do not seem to be predictors.  other features do.
# but R-squared is only .34?  

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [ ]:
# Your code here


In [ ]:
# Your comments here

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!